In [1]:
import os
import random
import re
import time
import warnings
import pandas as pd
import requests

In [25]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = "1NSkKVFpHZCihYMakWry6kFjuzyZ4TTSm"
destination = "../input/data.pkl"
if not os.path.exists(destination):
    download_file_from_google_drive(file_id, destination)

df = pd.read_pickle(destination)
# 3名中2名以上が同じラベルのものを真のラベル，全員ことなるラベルをつけていた場合にはラベルなしとする
df['resA_label'] = df[list(filter(lambda x: "resA" in x, df.columns))].apply(lambda d: sum(list(d),[]), axis=1).apply(lambda l: max(set(l), key = l.count) if len(set(l)) != len(l) else None)
df['resB_label'] = df[list(filter(lambda x: "resB" in x, df.columns))].apply(lambda d: sum(list(d),[]), axis=1).apply(lambda l: max(set(l), key = l.count) if len(set(l)) != len(l) else None)
display(df.head())

,id,user1_resA,user2_resA,user3_resA,user1_resB,user2_resB,user3_resB,user_id_list,textDisplay,media,resA_label,resB_label
0,370,[A2],[A2],[A2],[B3],[B2],[B4],"[13, 0, 12]",田口アホやな何が許せないの？町もすみませんといってきてるんだろ？勤め先も行ってきなって言うに...,youtube,A2,None
1,411,[A3],[A1],[A1],[B4],[B3],[B3],"[13, 0, 12]",町の責任で、若者の人生狂わせたとか同情の声があるが、自分は遅かれ早かれ犯罪は犯したと思う。人...,youtube,A1,B3
2,1281,[A2],[A2],[A2],[B3],[B2],[B1],"[13, 0, 12]",多分、殺されると思うけど、そうなれば、ますますプーチンの悪魔性が歴史と世界の人々の記憶に残る...,youtube,A2,None
3,1335,[A2],[A2],[A1],[B1],[B4],[B1],"[13, 0, 12]",広島・長崎がどうなったかロシアは知らないのか？知らねえわけないよな。核なんか使ってみろ、地獄...,youtube,A2,B1
4,1429,[A3],[A1],[A1],[B4],[B3],[B2],"[13, 0, 12]",もう遺憾砲しか言わないんだから諦めましょうよー国民が色々言っても今の政権は無視ですから(笑)...,youtube,A1,None


In [42]:
data = df.query("resA_label.notnull()&resB_label.notnull()&resA_label!='C'&resB_label!='C'")[["textDisplay", "resA_label", "resB_label"]].reset_index(drop=True)
display(data.head())
display(data.groupby("resB_label").count())
print(data.shape)

,textDisplay,resA_label,resB_label
0,町の責任で、若者の人生狂わせたとか同情の声があるが、自分は遅かれ早かれ犯罪は犯したと思う。人...,A1,B3
1,広島・長崎がどうなったかロシアは知らないのか？知らねえわけないよな。核なんか使ってみろ、地獄...,A2,B1
2,領空侵犯はしてないってどゆこと？尖閣諸島は日本の領土と認めちゃってるじゃん（笑）そこは、侵犯...,A3,B4
3,回収とか どうでもいい町長 誤送金した役人そもそもあんたらが原因👎どうして 公務員って当たり...,A1,B3
4,ダブルチェック、トリプルチェック！アホか！DX言われてるのに。頭つかってミスが発生しない仕組...,A3,B4


,textDisplay,resA_label
resB_label,,
B1,39,39
B2,123,123
B3,115,115
B4,613,613


(890, 3)


In [12]:
data = df[df['resA_label']!="None"][df['resB_label']!="None"][["textDisplay", "resA_label", "resB_label"]].reset_index(drop=True)
display(data.head())
print(data.shape)

/tmp/ipykernel_19300/1158033059.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = df[df['resA_label']!="None"][df['resB_label']!="None"][["textDisplay", "resA_label", "resB_label"]].reset_index(drop=True)


,textDisplay,resA_label,resB_label
0,町の責任で、若者の人生狂わせたとか同情の声があるが、自分は遅かれ早かれ犯罪は犯したと思う。人...,A1,B3
1,広島・長崎がどうなったかロシアは知らないのか？知らねえわけないよな。核なんか使ってみろ、地獄...,A2,B1
2,領空侵犯はしてないってどゆこと？尖閣諸島は日本の領土と認めちゃってるじゃん（笑）そこは、侵犯...,A3,B4
3,回収とか どうでもいい町長 誤送金した役人そもそもあんたらが原因👎どうして 公務員って当たり...,A1,B3
4,ダブルチェック、トリプルチェック！アホか！DX言われてるのに。頭つかってミスが発生しない仕組...,A3,B4


(898, 3)


In [5]:
# 機械学習の分類タスク
# とりあえずAtracked/Normalの2種類で分類する

In [6]:
data.groupby(["resA_label", "resB_label"]).count()

textDisplay
resA_label resB_label             
A1         B1                   18
           B2                   36
           B3                   68
           B4                  166
A2         B1                   16
           B2                   54
           B3                   36
           B4                  110
A3         B1                    5
           B2                   33
           B3                   11
           B4                  337
           C                     1
C          C                     7